In [0]:
# Decarbonization Trends Visualization
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, lag, expr
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [0]:
# Initialize Spark session (already available in Databricks)
spark = SparkSession.builder.appName("my_pudl_data.DecarbonizationTrends").getOrCreate()

In [0]:

# Option 1: If you've already created the decarbonization_trends table using the SQL query
decarbonization_df = spark.table("my_pudl_data.decarbonization_trends")

In [0]:
# Convert to Pandas DataFrame for visualization
pdf = decarbonization_df.toPandas()


In [0]:
# Add year-over-year changes
pdf = pdf.sort_values(['state', 'year'])
pdf['yoy_co2_change_tons'] = pdf.groupby('state')['total_co2_tons'].diff()
pdf['yoy_co2_change_pct'] = pdf.groupby('state')['total_co2_tons'].pct_change() * 100

In [0]:

# Calculate national averages for comparison
national_avg = pdf.groupby('year').agg({
    'co2_intensity_tons_per_mwh': 'mean',
    'renewable_capacity_fraction': 'mean',
    'total_co2_tons': 'sum',
    'coal_generation_mwh': 'sum',
    'gas_generation_mwh': 'sum',
    'nuclear_generation_mwh': 'sum',
    'renewable_generation_mwh': 'sum'
}).reset_index()

national_avg['total_generation_mwh'] = (
    national_avg['coal_generation_mwh'] + 
    national_avg['gas_generation_mwh'] + 
    national_avg['nuclear_generation_mwh'] + 
    national_avg['renewable_generation_mwh']
)

# Calculate generation percentages for national average
for fuel_type in ['coal', 'gas', 'nuclear', 'renewable']:
    national_avg[f'{fuel_type}_generation_pct'] = (
        national_avg[f'{fuel_type}_generation_mwh'] / national_avg['total_generation_mwh'] * 100
    )

# 1. CO2 Emissions Intensity Map by State (for most recent year)
most_recent_year = pdf['year'].max()
recent_data = pdf[pdf['year'] == most_recent_year]

# Create a choropleth map of emissions intensity
fig1 = px.choropleth(
    recent_data,
    locations='state',
    locationmode='USA-states',
    color='co2_intensity_tons_per_mwh',
    scope='usa',
    title=f'CO2 Emissions Intensity by State ({most_recent_year})',
    color_continuous_scale='Reds',
    labels={'co2_intensity_tons_per_mwh': 'CO2 Tons/MWh'}
)
display(fig1)

# 2. Top 10 States with Highest Renewable Capacity Fraction
top_renewable_states = recent_data.sort_values('renewable_capacity_fraction', ascending=False).head(10)

fig2 = px.bar(
    top_renewable_states,
    x='state',
    y='renewable_capacity_fraction',
    title=f'Top 10 States by Renewable Capacity Fraction ({most_recent_year})',
    labels={'renewable_capacity_fraction': 'Renewable Capacity Fraction', 'state': 'State'},
    color='renewable_capacity_fraction',
    color_continuous_scale='Greens'
)
display(fig2)

# 3. Time Series of CO2 Emissions for Selected States
selected_states = ['CA', 'TX', 'NY', 'FL', 'IL']  # Example states, can be modified
selected_states_data = pdf[pdf['state'].isin(selected_states)]

fig3 = px.line(
    selected_states_data,
    x='year',
    y='total_co2_tons',
    color='state',
    title='CO2 Emissions Over Time for Selected States',
    labels={'total_co2_tons': 'Total CO2 Emissions (Tons)', 'year': 'Year', 'state': 'State'}
)
display(fig3)

# 4. Emissions Intensity vs. Renewable Capacity Fraction Scatter Plot
fig4 = px.scatter(
    recent_data,
    x='renewable_capacity_fraction',
    y='co2_intensity_tons_per_mwh',
    size='total_co2_tons',
    color='state',
    hover_name='state',
    title=f'Emissions Intensity vs. Renewable Capacity ({most_recent_year})',
    labels={
        'renewable_capacity_fraction': 'Renewable Capacity Fraction',
        'co2_intensity_tons_per_mwh': 'CO2 Intensity (Tons/MWh)',
        'total_co2_tons': 'Total CO2 Emissions'
    }
)
display(fig4)

# 5. National Fuel Mix Trends Over Time
fig5 = make_subplots(specs=[[{"secondary_y": True}]])

fig5.add_trace(
    go.Bar(
        x=national_avg['year'],
        y=national_avg['coal_generation_pct'],
        name='Coal',
        marker_color='brown'
    )
)

fig5.add_trace(
    go.Bar(
        x=national_avg['year'],
        y=national_avg['gas_generation_pct'],
        name='Natural Gas',
        marker_color='gray'
    )
)

fig5.add_trace(
    go.Bar(
        x=national_avg['year'],
        y=national_avg['nuclear_generation_pct'],
        name='Nuclear',
        marker_color='purple'
    )
)

fig5.add_trace(
    go.Bar(
        x=national_avg['year'],
        y=national_avg['renewable_generation_pct'],
        name='Renewable',
        marker_color='green'
    )
)

fig5.add_trace(
    go.Scatter(
        x=national_avg['year'],
        y=national_avg['co2_intensity_tons_per_mwh'],
        name='CO2 Intensity',
        line=dict(color='red', width=3),
        mode='lines+markers'
    ),
    secondary_y=True
)

fig5.update_layout(
    title='National Generation Mix and CO2 Intensity Trends',
    barmode='stack',
    xaxis_title='Year',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig5.update_yaxes(title_text="Generation Percentage", secondary_y=False)
fig5.update_yaxes(title_text="CO2 Intensity (Tons/MWh)", secondary_y=True)

display(fig5)

# 6. Year-over-Year CO2 Reduction Leaders (most recent year transition)
recent_yoy = pdf[(pdf['year'] == most_recent_year) & (pdf['yoy_co2_change_pct'] < 0)]
top_reducers = recent_yoy.sort_values('yoy_co2_change_pct').head(10)

fig6 = px.bar(
    top_reducers,
    x='state',
    y='yoy_co2_change_pct',
    title=f'Top 10 States by CO2 Emissions Reduction ({most_recent_year-1} to {most_recent_year})',
    labels={'yoy_co2_change_pct': 'Year-over-Year CO2 Reduction (%)', 'state': 'State'},
    color='yoy_co2_change_pct',
    color_continuous_scale='Blues'
)
display(fig6)

# 7. Create a dashboard of state-specific decarbonization trends
# Function to create state dashboard
def create_state_dashboard(state_code):
    state_data = pdf[pdf['state'] == state_code].sort_values('year')
    
    # Create subplot figure
    state_fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f'CO2 Emissions Over Time in {state_code}',
            f'Generation Mix in {state_code}',
            f'Emissions Intensity in {state_code}',
            f'Renewable Capacity Fraction in {state_code}'
        ),
        specs=[[{"type": "scatter"}, {"type": "bar"}],
               [{"type": "scatter"}, {"type": "scatter"}]]
    )
    
    # 1. CO2 Emissions Over Time
    state_fig.add_trace(
        go.Scatter(x=state_data['year'], y=state_data['total_co2_tons'], mode='lines+markers'),
        row=1, col=1
    )
    
    # 2. Generation Mix
    for i, fuel_type in enumerate(['coal', 'gas', 'nuclear', 'renewable']):
        colors = ['brown', 'gray', 'purple', 'green']
        state_fig.add_trace(
            go.Bar(
                x=state_data['year'], 
                y=state_data[f'{fuel_type}_generation_mwh'],
                name=fuel_type.capitalize(),
                marker_color=colors[i]
            ),
            row=1, col=2
        )
    
    # 3. Emissions Intensity
    state_fig.add_trace(
        go.Scatter(
            x=state_data['year'], 
            y=state_data['co2_intensity_tons_per_mwh'], 
            mode='lines+markers',
            line=dict(color='red')
        ),
        row=2, col=1
    )
    
    # 4. Renewable Capacity Fraction
    state_fig.add_trace(
        go.Scatter(
            x=state_data['year'], 
            y=state_data['renewable_capacity_fraction'], 
            mode='lines+markers',
            line=dict(color='green')
        ),
        row=2, col=2
    )
    
    state_fig.update_layout(
        title=f'Decarbonization Dashboard for {state_code}',
        height=800,
        showlegend=False
    )
    
    return state_fig

# Create dashboards for a few example states
for state_code in ['CA', 'TX', 'NY']:
    state_dashboard = create_state_dashboard(state_code)
    display(state_dashboard)

# Create a dropdown widget to select different states for the dashboard
from ipywidgets import widgets
from IPython.display import display as ipydisplay

state_dropdown = widgets.Dropdown(
    options=sorted(pdf['state'].unique()),
    value='CA',
    description='Select State:',
    style={'description_width': 'initial'}
)

def on_state_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        state_dashboard = create_state_dashboard(change['new'])
        display(state_dashboard)

state_dropdown.observe(on_state_change, names='value')
ipydisplay(state_dropdown)

# 8. Decarbonization Progress Index
# Create a composite score based on multiple factors
recent_data['emissions_intensity_score'] = (
    1 - (recent_data['co2_intensity_tons_per_mwh'] / recent_data['co2_intensity_tons_per_mwh'].max())
) * 40  # 40% weight

recent_data['renewable_capacity_score'] = (
    recent_data['renewable_capacity_fraction'] / recent_data['renewable_capacity_fraction'].max()
) * 30  # 30% weight

recent_data['emissions_reduction_score'] = (
    recent_data['yoy_co2_change_pct'].clip(upper=0) / recent_data['yoy_co2_change_pct'].min()
) * 30  # 30% weight

# Calculate total decarbonization score
recent_data['decarbonization_score'] = (
    recent_data['emissions_intensity_score'].fillna(0) + 
    recent_data['renewable_capacity_score'].fillna(0) + 
    recent_data['emissions_reduction_score'].fillna(0)
)

# Display top decarbonization performers
top_performers = recent_data.sort_values('decarbonization_score', ascending=False).head(10)

fig8 = px.bar(
    top_performers,
    x='state',
    y='decarbonization_score',
    title=f'Top 10 States by Decarbonization Progress ({most_recent_year})',
    labels={'decarbonization_score': 'Decarbonization Score', 'state': 'State'},
    color='decarbonization_score',
    color_continuous_scale='Viridis'
)
display(fig8)

# Save the processed data to a new table for future use
spark.createDataFrame(pdf).write.mode("overwrite").saveAsTable("decarbonization_trends_processed")

print("Decarbonization visualization and analysis complete!")